## 0- Objectif du notebook

Ce notebook vise à afficher sous folium l'évolution des incdendies par état, avec l'aide d'un slider et l'utilisation d'une échelle de couleurs.

## 1- Chargement des librairies et du jeu de données initial

In [1]:
import pandas as pd
import numpy as np
import folium
import geoplot as gplt
import geopandas as gpd
import geoplot.crs as gcrs

import warnings
warnings.filterwarnings('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\geopandas\_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
df = pd.read_csv('Fires_filter.csv', index_col = 'OBJECTID')
df.DISCOVERY_DATE = pd.to_datetime(df.DISCOVERY_DATE)
df.CONT_DATE = pd.to_datetime(df.CONT_DATE)

Nous allons travailler 3 cartes Folium permettant de suivre les métriques suivantes :
- Le nombre cumulé de feux par état
- La surface brûlée cumulée par état
- Le pourcentage cumulé du territoire brûlé par état

## 2a- Evol du nombre de feux : préparation des données

Chargement de la carte des US au format Geopandas

In [3]:
usa = gpd.read_file("../maps/cb_2020_us_state_20m.shp")

Constitution du jeu de données de travail avec le décompte de feux par année et par état

In [4]:
#df_mega_count = df.query("FIRE_SIZE_CLASS == 'G'").groupby(['FIRE_YEAR', 'STATE'], as_index = False)[['FIRE_SIZE_CLASS']].count()
df_mega_count = df.groupby(['FIRE_YEAR', 'STATE'], as_index = False)[['FIRE_SIZE_CLASS']].count()

Ajout d'une colonne de sommme cumulée du nombre de feux reportés par année et par état

In [5]:
df_mega_count['cumsum'] = df_mega_count.FIRE_SIZE_CLASS

for i in df_mega_count.STATE.unique().tolist():
    list_index = df_mega_count[df_mega_count.STATE == i].index
    base = df_mega_count.iloc[list_index[0], 2]
    for j in list_index[1:]:
        df_mega_count.iloc[j, 3] = base + df_mega_count.iloc[j, 3]
        base = df_mega_count.iloc[j, 3]

Merge avec les données de l'objet Geopandas afin d'avoir le nom des états

In [6]:
df_mega_count = df_mega_count.merge(usa, left_on = 'STATE', right_on = 'STUSPS')

Transformation de la donnée "année" en nombre de secondes depuis l'epoch

In [7]:
df_mega_count['date_sec'] = pd.to_datetime(df_mega_count['FIRE_YEAR'], format = '%Y').apply(lambda x : pd.Timestamp(x).timestamp()).astype(int)

## 2b- Evol des surfaces brûlées : préparation des données

Chargement de la carte des US au format Geopandas

In [3]:
usa = gpd.read_file("../maps/cb_2020_us_state_20m.shp")

Constitution du jeu de données de travail avec le décompte de feux par année et par état

In [4]:
#df_mega_count = df.query("FIRE_SIZE_CLASS == 'G'").groupby(['FIRE_YEAR', 'STATE'], as_index = False)[['FIRE_SIZE_CLASS']].count()
df_mega_surf = df.groupby(['FIRE_YEAR', 'STATE'], as_index = False)[['FIRE_SIZE']].sum()

Ajout d'une colonne de sommme cumulée du nombre de feux reportés par année et par état

In [5]:
df_mega_surf['cumsum'] = df_mega_surf.FIRE_SIZE

for i in df_mega_surf.STATE.unique().tolist():
    list_index = df_mega_surf[df_mega_surf.STATE == i].index
    base = df_mega_surf.iloc[list_index[0], 2]
    for j in list_index[1:]:
        df_mega_surf.iloc[j, 3] = base + df_mega_surf.iloc[j, 3]
        base = df_mega_surf.iloc[j, 3]

df_mega_surf['cumsum'] = np.round(df_mega_surf['cumsum']*0.00404686, 0)
df_mega_surf['cumsum'] = df_mega_surf['cumsum'].astype(int)

In [6]:
df_mega_surf

,FIRE_YEAR,STATE,FIRE_SIZE,cumsum
0,1992,AK,142717.400,578
1,1992,AL,1253.200,5
2,1992,AR,1478.500,6
3,1992,AZ,42593.650,172
4,1992,CA,296439.800,1200
...,...,...,...,...
1209,2015,VT,346.429,6
1210,2015,WA,1161646.380,19353
1211,2015,WI,2721.440,359
1212,2015,WV,17251.370,2334


Merge avec les données de l'objet Geopandas afin d'avoir le nom des états

In [7]:
df_mega_surf = df_mega_surf.merge(usa, left_on = 'STATE', right_on = 'STUSPS')

Transformation de la donnée "année" en nombre de secondes depuis l'epoch

In [8]:
df_mega_surf['date_sec'] = pd.to_datetime(df_mega_surf['FIRE_YEAR'], format = '%Y').apply(lambda x : pd.Timestamp(x).timestamp()).astype(int)

## 2c- Evol du ratio de surfaces brûlées : préparation des données

Chargement de la carte des US au format Geopandas

In [3]:
usa = gpd.read_file("../maps/cb_2020_us_state_20m.shp")

Import données de surface de chaque état

In [4]:
us_area = pd.read_csv('US_area.csv', sep = ";", names = ['STATE', 'AREA'])

In [5]:
us_area.AREA = np.round(us_area.AREA * 2.58999, 0).astype(int)
us_area.STATE = us_area.STATE.apply(lambda x: x.replace(" ", ""))
us_area.set_index('STATE', inplace = True)

In [60]:
us_area.sample(5)

,AREA
STATE,
KS,131681
FL,86900
KY,63942
NM,195318
WI,87410


In [9]:
us_area.drop('DC', axis = 0, inplace = True)

In [10]:
us_area.loc['DC'] = 177

In [11]:
us_area.loc['PR'] = 9104

Constitution du jeu de données de travail avec le décompte de feux par année et par état

In [12]:
df_mega_surf = df.groupby(['FIRE_YEAR', 'STATE'], as_index = False)[['FIRE_SIZE']].sum()

In [69]:
df_mega_surf

,FIRE_YEAR,STATE,FIRE_SIZE
0,1992,AK,142717.400
1,1992,AL,1253.200
2,1992,AR,1478.500
3,1992,AZ,42593.650
4,1992,CA,296439.800
...,...,...,...
1209,2015,VT,346.429
1210,2015,WA,1161646.380
1211,2015,WI,2721.440
1212,2015,WV,17251.370


Ajout d'une colonne de sommme cumulée du nombre de feux reportés par année et par état

In [13]:
df_mega_surf.FIRE_SIZE = df_mega_surf.FIRE_SIZE * 0.00404686

In [14]:
df_mega_surf['cumratio'] = df_mega_surf.FIRE_SIZE

for i in df_mega_surf.STATE.unique().tolist():
    list_index = df_mega_surf[df_mega_surf.STATE == i].index
    base = df_mega_surf.iloc[list_index[0], 2]
    df_mega_surf.iloc[list_index[0], 3] = base / us_area.loc[i]
    for j in list_index[1:]:
        df_mega_surf.iloc[j, 3] = (base + df_mega_surf.iloc[j, 3])
        base = df_mega_surf.iloc[j, 3]
        df_mega_surf.iloc[j, 3] = (base + df_mega_surf.iloc[j, 3]) / us_area.loc[i]
        
df_mega_surf['cumratio'] = np.round(df_mega_surf['cumratio'] * 100, 2)

In [124]:
df_mega_surf

,FIRE_YEAR,STATE,FIRE_SIZE,cumratio,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry,date_sec,colour
0,1992,AK,577.557337,0.06,02,01785533,0400000US02,02,AK,Alaska,00,1478941109938,245380162784,"MULTIPOLYGON (((179.48132 51.97530, 179.58286 ...",694224000,#ffffcaff
1,1993,AK,2780.967794,0.73,02,01785533,0400000US02,02,AK,Alaska,00,1478941109938,245380162784,"MULTIPOLYGON (((179.48132 51.97530, 179.58286 ...",725846400,#fffbc1ff
2,1994,AK,1060.135275,0.96,02,01785533,0400000US02,02,AK,Alaska,00,1478941109938,245380162784,"MULTIPOLYGON (((179.48132 51.97530, 179.58286 ...",757382400,#fffec9ff
3,1995,AK,177.908464,1.00,02,01785533,0400000US02,02,AK,Alaska,00,1478941109938,245380162784,"MULTIPOLYGON (((179.48132 51.97530, 179.58286 ...",788918400,#ffffccff
4,1996,AK,2423.248437,1.53,02,01785533,0400000US02,02,AK,Alaska,00,1478941109938,245380162784,"MULTIPOLYGON (((179.48132 51.97530, 179.58286 ...",820454400,#fffcc4ff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,2011,RI,0.113474,0.18,44,01219835,0400000US44,44,RI,Rhode Island,00,2677759219,1323691129,"MULTIPOLYGON (((-71.63147 41.16668, -71.59334 ...",1293840000,#ffffccff
1210,2012,RI,0.161510,0.19,44,01219835,0400000US44,44,RI,Rhode Island,00,2677759219,1323691129,"MULTIPOLYGON (((-71.63147 41.16668, -71.59334 ...",1325376000,#ffffccff
1211,2013,RI,0.067583,0.20,44,01219835,0400000US44,44,RI,Rhode Island,00,2677759219,1323691129,"MULTIPOLYGON (((-71.63147 41.16668, -71.59334 ...",1356998400,#ffffccff
1212,2014,RI,0.094292,0.21,44,01219835,0400000US44,44,RI,Rhode Island,00,2677759219,1323691129,"MULTIPOLYGON (((-71.63147 41.16668, -71.59334 ...",1388534400,#ffffccff


In [15]:
df_mega_surf.cumratio.max()

56.68

In [126]:
df_mega_surf[df_mega_surf.STATE ==  'TX']

,FIRE_YEAR,STATE,FIRE_SIZE,cumratio,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry,date_sec,colour
1015,1992,TX,136.016179,0.03,48,01779801,0400000US48,48,TX,Texas,00,676680588914,18979352230,"POLYGON ((-106.62345 31.91403, -106.63011 31.9...",694224000,#ffffcbff
1016,1993,TX,472.971502,0.29,48,01779801,0400000US48,48,TX,Texas,00,676680588914,18979352230,"POLYGON ((-106.62345 31.91403, -106.63011 31.9...",725846400,#fffec8ff
1017,1994,TX,286.993194,0.43,48,01779801,0400000US48,48,TX,Texas,00,676680588914,18979352230,"POLYGON ((-106.62345 31.91403, -106.63011 31.9...",757382400,#ffffcaff
1018,1995,TX,228.125950,0.53,48,01779801,0400000US48,48,TX,Texas,00,676680588914,18979352230,"POLYGON ((-106.62345 31.91403, -106.63011 31.9...",788918400,#ffffcbff
1019,1996,TX,875.024210,0.95,48,01779801,0400000US48,48,TX,Texas,00,676680588914,18979352230,"POLYGON ((-106.62345 31.91403, -106.63011 31.9...",820454400,#fffdc6ff
1020,1997,TX,189.315753,1.04,48,01779801,0400000US48,48,TX,Texas,00,676680588914,18979352230,"POLYGON ((-106.62345 31.91403, -106.63011 31.9...",852076800,#ffffcbff
1021,1998,TX,552.604399,1.30,48,01779801,0400000US48,48,TX,Texas,00,676680588914,18979352230,"POLYGON ((-106.62345 31.91403, -106.63011 31.9...",883612800,#fffec8ff
1022,1999,TX,242.925721,1.42,48,01779801,0400000US48,48,TX,Texas,00,676680588914,18979352230,"POLYGON ((-106.62345 31.91403, -106.63011 31.9...",915148800,#ffffcaff
1023,2000,TX,735.261043,1.76,48,01779801,0400000US48,48,TX,Texas,00,676680588914,18979352230,"POLYGON ((-106.62345 31.91403, -106.63011 31.9...",946684800,#fffdc7ff
1024,2001,TX,373.226520,1.94,48,01779801,0400000US48,48,TX,Texas,00,676680588914,18979352230,"POLYGON ((-106.62345 31.91403, -106.63011 31.9...",978307200,#fffecaff


Merge avec les données de l'objet Geopandas afin d'avoir le nom des états

In [16]:
df_mega_surf = df_mega_surf.merge(usa, left_on = 'STATE', right_on = 'STUSPS')

Transformation de la donnée "année" en nombre de secondes depuis l'epoch

In [17]:
df_mega_surf['date_sec'] = pd.to_datetime(df_mega_surf['FIRE_YEAR'], format = '%Y').apply(lambda x : pd.Timestamp(x).timestamp()).astype(int)

In [136]:
df_mega_surf.iloc[df_mega_surf.cumratio.nlargest(10).index]

,FIRE_YEAR,STATE,FIRE_SIZE,cumratio,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry,date_sec,colour
47,2015,AL,124.779708,91.23,01,01779775,0400000US01,01,AL,Alabama,00,131175460655,4591915034,"POLYGON ((-88.46866 31.89386, -88.46866 31.933...",1420070400,#800026ff
143,2015,CO,92.277918,89.32,08,01779779,0400000US08,08,CO,Colorado,00,268418746964,1185766400,"POLYGON ((-109.05996 38.49999, -109.05151 39.1...",1420070400,#8a0026ff
142,2014,CO,99.934415,88.21,08,01779779,0400000US08,08,CO,Colorado,00,268418746964,1185766400,"POLYGON ((-109.05996 38.49999, -109.05151 39.1...",1388534400,#900026ff
46,2014,AL,153.624896,88.17,01,01779775,0400000US01,01,AL,Alabama,00,131175460655,4591915034,"POLYGON ((-88.46866 31.89386, -88.46866 31.933...",1388534400,#900026ff
141,2013,CO,656.118343,87.02,08,01779779,0400000US08,08,CO,Colorado,00,268418746964,1185766400,"POLYGON ((-109.05996 38.49999, -109.05151 39.1...",1356998400,#970026ff
45,2013,AL,121.144413,84.41,01,01779775,0400000US01,01,AL,Alabama,00,131175460655,4591915034,"POLYGON ((-88.46866 31.89386, -88.46866 31.933...",1356998400,#a50026ff
323,2015,ID,3201.171924,83.17,16,01779783,0400000US16,16,ID,Idaho,00,214049923496,2391577745,"POLYGON ((-117.24303 44.39097, -117.21507 44.4...",1420070400,#ab0026ff
44,2012,AL,117.992112,81.44,01,01779775,0400000US01,01,AL,Alabama,00,131175460655,4591915034,"POLYGON ((-88.46866 31.89386, -88.46866 31.933...",1325376000,#b50026ff
140,2012,CO,1097.764962,79.16,08,01779779,0400000US08,08,CO,Colorado,00,268418746964,1185766400,"POLYGON ((-109.05996 38.49999, -109.05151 39.1...",1325376000,#bf0125ff
43,2011,AL,218.923714,78.55,01,01779775,0400000US01,01,AL,Alabama,00,131175460655,4591915034,"POLYGON ((-88.46866 31.89386, -88.46866 31.933...",1293840000,#c20225ff


## 3a- Evol du nombre de feux

Mise en place d'une échelle de couleurs et association d'une teinte à une valeur donnée de cumsum

In [9]:
import branca.colormap as cm

max_colour = max(df_mega_count['cumsum'])
min_colour = min(df_mega_count['cumsum'])
#cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
cmap = cm.linear.Reds_09.scale(min_colour, max_colour)
df_mega_count['colour'] = df_mega_count['cumsum'].map(cmap)

NameError: name 'df_mega_count' is not defined

Définition d'un dictionnaire associant à chaque ID d'état, les timestamps et les teintes de couleurs

In [15]:
state_list = df_mega_count['NAME'].unique().tolist()
state_idx = range(len(state_list))

style_dict = {}
for i in state_idx:
    state = state_list[i]
    result = df_mega_count[df_mega_count['NAME'] == state]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['date_sec']] = {'color': r['colour'], 'opacity': 0.7}
    style_dict[str(i)] = inner_dict

Définition d'un objet Geopandas pour les contours des états

In [16]:
states_df = df_mega_count[['geometry']]
states_gdf = gpd.GeoDataFrame(states_df)
states_gdf = states_gdf.drop_duplicates().reset_index()

Affichage de la carte avec le TimeSlider

In [23]:
from folium.plugins import TimeSliderChoropleth

slider_map = folium.Map(location=[40, -95], zoom_start=4)

_ = TimeSliderChoropleth(
    data=states_gdf.to_json(),
    styledict=style_dict,

).add_to(slider_map)

_ = cmap.add_to(slider_map)
cmap.caption = "Nombre cumulé de feux reportés par état (1992-2015)"

slider_map

## 3a- Evol des surfaces brûlées

Mise en place d'une échelle de couleurs et association d'une teinte à une valeur donnée de cumsum

In [127]:
import branca.colormap as cm

max_colour = max(df_mega_surf['cumsum'])
min_colour = min(df_mega_surf['cumsum'])
cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
df_mega_surf['colour'] = df_mega_surf['cumsum'].map(cmap)

KeyError: 'cumsum'

Définition d'un dictionnaire associant à chaque ID d'état, les timestamps et les teintes de couleurs

In [11]:
state_list = df_mega_surf['NAME'].unique().tolist()
state_idx = range(len(state_list))

style_dict = {}
for i in state_idx:
    state = state_list[i]
    result = df_mega_surf[df_mega_surf['NAME'] == state]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['date_sec']] = {'color': r['colour'], 'opacity': 0.7}
    style_dict[str(i)] = inner_dict

Définition d'un objet Geopandas pour les contours des états

In [12]:
states_df = df_mega_surf[['geometry']]
states_gdf = gpd.GeoDataFrame(states_df)
states_gdf = states_gdf.drop_duplicates().reset_index()

Affichage de la carte avec le TimeSlider

In [13]:
from folium.plugins import TimeSliderChoropleth

slider_map = folium.Map(location=[40, -95], zoom_start=4)

_ = TimeSliderChoropleth(
    data=states_gdf.to_json(),
    styledict=style_dict,

).add_to(slider_map)

_ = cmap.add_to(slider_map)
cmap.caption = "Surfaces brûlées cumulées en km², par état, de 1992 à 2015"

slider_map

## 3- Evol du pourcentage du territoire brûlé

Mise en place d'une échelle de couleurs et association d'une teinte à une valeur donnée de cumsum

In [18]:
import branca.colormap as cm

max_colour = max(df_mega_surf['cumratio'])
min_colour = min(df_mega_surf['cumratio'])
cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
df_mega_surf['colour'] = df_mega_surf['cumratio'].map(cmap)

Définition d'un dictionnaire associant à chaque ID d'état, les timestamps et les teintes de couleurs

In [19]:
state_list = df_mega_surf['NAME'].unique().tolist()
state_idx = range(len(state_list))

style_dict = {}
for i in state_idx:
    state = state_list[i]
    result = df_mega_surf[df_mega_surf['NAME'] == state]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['date_sec']] = {'color': r['colour'], 'opacity': 0.7}
    style_dict[str(i)] = inner_dict

Définition d'un objet Geopandas pour les contours des états

In [20]:
states_df = df_mega_surf[['geometry']]
states_gdf = gpd.GeoDataFrame(states_df)
states_gdf = states_gdf.drop_duplicates().reset_index()

Affichage de la carte avec le TimeSlider

In [21]:
from folium.plugins import TimeSliderChoropleth

slider_map = folium.Map(location=[40, -95], zoom_start=4)

_ = TimeSliderChoropleth(
    data=states_gdf.to_json(),
    styledict=style_dict,

).add_to(slider_map)

_ = cmap.add_to(slider_map)
cmap.caption = "Pourcentage cumulé de la surface brûlé, par état, de 1992 à 2015"

slider_map